文本抄袭自动检测分析:
如果你是某新闻单位工作人员（这里假设source=新华社），为了防止其他媒体抄袭你的文章，你打算做一个抄袭自动检测分析的工具
1）定义可能抄袭的文章来源
2）与原文对比定位抄袭的地方
原始数据：sqlResult.csv，共计89611篇
从数据库导出的文章，字段包括：id, author, source, content, feature, title, url
常用中文停用词：chinese_stopwords.txt

"1、完成聚类算法（10points）
2、完成分类算法（10points）
3、完成相似度计算，Top-N排序（10points）
4、完成代码，结果正确（30points）"

Step1:数据加载,加载sqlResult.csv及停用词chinese_stopwords.txt

In [1]:
import pandas as pd
import numpy as np
import jieba

In [2]:
news = pd.read_csv("sqlResult.csv", encoding="gb18030")
print(news.shape)
print(news.head())

(89611, 7)
      id      author                  source  \
0  89617         NaN  快科技@http://www.kkj.cn/   
1  89616         NaN  快科技@http://www.kkj.cn/   
2  89615         NaN  快科技@http://www.kkj.cn/   
3  89614         NaN                     新华社   
4  89613  胡淑丽_MN7479                   深圳大件事   

                                             content  \
0  此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...   
1  骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...   
2  此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...   
3    这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n   
4  （原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...   

                                             feature  \
0  {"type":"科技","site":"cnbeta","commentNum":"37"...   
1  {"type":"科技","site":"cnbeta","commentNum":"15"...   
2  {"type":"科技","site":"cnbeta","commentNum":"18"...   
3  {"type":"国际新闻","site":"环球","commentNum":"0","j...   
4  {"type":"新闻","site":"网易热门","commentNum":"978",...   

                          

In [3]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89611 entries, 0 to 89610
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       89611 non-null  int64 
 1   author   79396 non-null  object
 2   source   89609 non-null  object
 3   content  87054 non-null  object
 4   feature  89611 non-null  object
 5   title    89577 non-null  object
 6   url      87144 non-null  object
dtypes: int64(1), object(6)
memory usage: 4.8+ MB


In [4]:
news = news.dropna(subset=['content'])
news.shape

(87054, 7)

In [5]:
# 加载停用词
with open('chinese_stopwords.txt', 'r', encoding='utf-8') as file:
    stopwords = [i[:-1] for i in file.readlines()]
stopwords[:5]

['，', '的', '。', '、', '在']

In [6]:
# 分词
def split_text(text):
    text = text.replace(' ','').replace('\n','')
    text2 = jieba.cut(text)
    result = ' '.join([w for w in text2 if w not in stopwords])
    return result

In [7]:
print(news.iloc[0].content)

此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。
MIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。
当然，关于MIUI 9的确切信息，我们还是等待官方消息。



In [8]:
split_text(news.iloc[0].content)

Building prefix dict from D:\python_soft\anaconda3\lib\site-packages\jieba\dict.txt ...
Loading model from cache C:\Users\刘文明\AppData\Local\Temp\jieba.cache
Loading model cost 1.5191609859466553 seconds.
Prefix dict has been built succesfully.


'此外 本周 除 小米 手机 款 机型 外 机型 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 确保 工程师 集中 全部 精力 进行 系统优化 工作 有人 猜测 精力 主要 用到 MIUI9 研发 之中 \r MIUI8 去年 发布 距今已有 一年 有余 更新换代 \r 当然 MIUI9 确切 信息 等待 官方消息 \r'

In [9]:
corpus = list(map(split_text, [str(i) for i in news.content]))

In [10]:
# 计算corpus的TFIDF
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [11]:
countvectorizer = CountVectorizer(encoding='utf-8', min_df=0.015)
tfidftransfomer = TfidfTransformer()
counvector = countvectorizer.fit_transform(corpus)
tfidf = tfidftransfomer.fit_transform(counvector)

In [12]:
tfidf.shape

(87054, 884)

In [13]:
tfidf.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.35722359,
        0.        ]])

In [14]:
# 标记是否为自己的新闻
label = list(map(lambda source: 1 if '新华社' in str(source) else 0, news.source))
label[:10]

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]

In [15]:
# 数据集切分
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tfidf.toarray(), label, test_size=0.3)

In [16]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
y_predict = model.predict(X_test)
print('准确率:', accuracy_score(y_test, y_predict))
print('精确率:', precision_score(y_test, y_predict))
print('召回率:', recall_score(y_test, y_predict))

准确率: 0.8800015315694758
精确率: 0.9605026121419564
召回率: 0.9042995251017639


In [18]:
# 使用模型进行风格预测
prediction = model.predict(tfidf.toarray())
labels = np.array(label)
compare_news = pd.DataFrame({'prediction':prediction, 'labels':labels})

In [19]:
copy_news_index = compare_news[(compare_news['prediction']==1)&(compare_news['labels']==0)].index

In [20]:
# 实际为新华社的数量
xinhuashe_news_index = compare_news[(compare_news['labels']==1)].index
print('可能为copy的新闻条数：', len(copy_news_index))

可能为copy的新闻条数： 2811


In [21]:
from sklearn.preprocessing import Normalizer
from sklearn.cluster import  KMeans
normalizer = Normalizer()
scaled_array = normalizer.fit_transform(tfidf.toarray())

In [22]:
kmeans = KMeans(n_clusters=25)
k_labels = kmeans.fit_predict(scaled_array)
k_labels.shape

(87054,)

In [24]:
# 创建id_class
id_class = {index:class_ for index, class_ in enumerate(k_labels)}
from collections import defaultdict
class_id = defaultdict(set)
for index, class_ in id_class.items():
    if index in xinhuashe_news_index.tolist():
        class_id[class_].add(index)

In [31]:
from sklearn.metrics.pairwise import cosine_similarity
def find_similar_text(cpindex, top=10):
    dist_dict = {i:cosine_similarity(tfidf[cpindex], tfidf[i]) for i in class_id[id_class[cpindex]]}
    return sorted(dist_dict.items(), key=lambda x:x[1][0], reverse=True)[:top]

In [32]:
cpindex = 3352
print('是否再新华社', cpindex in xinhuashe_news_index)
print('是否在copy_news', cpindex in copy_news_index)

是否再新华社 False
是否在copy_news True


In [33]:
similar_list = find_similar_text(cpindex)
print(similar_list)

[(3134, array([[0.96849134]])), (63511, array([[0.94643198]])), (29441, array([[0.94283416]])), (3218, array([[0.87621892]])), (29615, array([[0.86936328]])), (29888, array([[0.86215862]])), (64046, array([[0.85278222]])), (29777, array([[0.84875422]])), (63974, array([[0.73415212]])), (63975, array([[0.73415212]]))]


In [37]:
print('怀疑抄袭：',news.iloc[cpindex].content)
similar1 = similar_list[0][0]
print('相似的原文：',news.iloc[similar1].content)

怀疑抄袭： 　　中国5月份56座城市新建商品住宅价格环比上涨，4月份为58座上涨。5月份15个一线和热点二线城市房地产市场基本稳定，5月份房地产调控政策效果继续显现。
　　统计局：15个一线和热点二线城市房价同比涨幅全部回落
　　国家统计局城市司高级统计师刘建伟解读5月份房价数据
　　5月份一二线城市房价平均涨幅继续回落
　　国家统计局今日发布了2017年5月份70个大中城市住宅销售价格统计数据。对此，国家统计局城市司高级统计师刘建伟进行了解读。
　　一、15个一线和热点二线城市新建商品住宅价格同比涨幅全部回落、9个城市环比下降或持平
　　5月份，因地制宜、因城施策的房地产调控政策效果继续显现，15个一线和热点二线城市房地产市场基本稳定。从同比看，15个城市新建商品住宅价格涨幅均比上月回落，回落幅度在0.5至6.4个百分点之间。从环比看，9个城市新建商品住宅价格下降或持平；5个城市涨幅在0.5%以内。
　　二、70个大中城市中一二线城市房价同比涨幅持续回落
　　5月份，70个城市中新建商品住宅和二手住宅价格同比涨幅比上月回落的城市分别有29和18个。其中，一二线城市同比涨幅回落尤其明显。据测算，一线城市新建商品住宅和二手住宅价格同比涨幅均连续8个月回落，5月份比4月份分别回落2.2和1.7个百分点；二线城市新建商品住宅和二手住宅价格同比涨幅分别连续6个月和4个月回落，5月份比4月份分别回落0.8和0.5个百分点。
　　三、70个大中城市中房价环比下降及涨幅回落城市个数均有所增加
　　5月份，70个城市中新建商品住宅价格环比下降的城市有9个，比上月增加1个；涨幅回落的城市有26个，比上月增加3个。二手住宅价格环比下降的城市有7个，比上月增加2个；涨幅回落的城市有30个，比上月增加8个。

相似的原文： 　　国家统计局19日发布数据，5月份，15个一线和热点二线城市新建商品住宅价格同比涨幅全部回落，其中9个城市环比下降或持平。这9个价格环比下降或持平的城市为：北京、上海、南京、杭州、合肥、福州、郑州、深圳、成都。
　　“5月份，因地制宜、因城施策的房地产调控政策效果继续显现，15个一线和热点二线城市房地产市场基本稳定。”国家统计局城市司高级统计师刘建伟说，从同比看，15个城市新建商品住宅价格涨幅均比上月回落，回落幅度在0.5至6.4个百分点之间。从环比看，9个城市新建商